In [13]:
!python -m jupytools export -nb "04_dense_net.ipynb" -o .

Exported: 04_dense_net.ipynb -> dense_net.py
1 notebook(s) exported into folder: .


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
#export
from collections import OrderedDict
import json
import os
from os.path import dirname, join
from pdb import set_trace

import jupytools
import jupytools.syspath
import numpy as np
import pandas as pd

jupytools.syspath.add(join(dirname(os.getcwd()), 'protein_project'))
jupytools.syspath.add('rxrx1-utils')
if jupytools.is_notebook():
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm as tdqm
    
import contextlib, io
with contextlib.redirect_stderr(io.StringIO()):
    from augmentation import Augmented, multichannel_norm
    from basedir import ROOT, NUM_CLASSES
    from catalyst_visdom import BatchMetricsPlotCallback, EpochMetricsPlotCallback
    from dataset import load_data, RxRxDataset
    from model.utils import freeze_all, unfreeze_layers
    from visual import rgb, six, show_1, show

In [4]:
#export
import albumentations as T

from catalyst.contrib.schedulers import OneCycleLR
from catalyst.contrib.modules import GlobalConcatPool2d
from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import AccuracyCallback, EarlyStoppingCallback
from catalyst.dl.core import Callback
from catalyst.utils import get_one_hot

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.data import DataLoader

import pretrainedmodels

In [5]:
#export
seed = 1
np.random.seed(seed)

In [6]:
#export
data_dict = load_data()
trn_rec, tst_rec = data_dict['data']
num_classes = data_dict['num_classes']

In [7]:
#export
with open('exp_channelwise_stats.json') as f:
    stats = json.load(f)

experiments = np.unique([r['experiment'] for r in trn_rec])
cell_types = np.unique([r['cell_type'] for r in trn_rec])
holdouts = []
for ct in cell_types:
    holdout = np.random.choice(experiments[
        [exp.startswith(ct) for exp in experiments]])
    holdouts.append(holdout)
    
train_exp, valid_exp = [exp for exp in experiments if exp not in holdouts], holdouts
print(f'Train experiments: {train_exp}')
print(f'Valid experiments: {valid_exp}')
train = [r for r in trn_rec if r['experiment'] in train_exp]
valid = [r for r in trn_rec if r['experiment'] in valid_exp]
print(f'Train size: {len(train)}, valid size: {len(valid)}')

transforms = T.Compose(
    additional_targets={f'image{i}': 'image' for i in range(5)},
    transforms=[
        T.Resize(224, 224), 
        T.VerticalFlip(p=0.25),
        T.HorizontalFlip(p=0.25)
    ]
)

batch_size = 512

trn_ds = Augmented(
    RxRxDataset(train, NUM_CLASSES, onehot=True), 
    tr=transforms,
    norm_fn=multichannel_norm,
    norm_stats=stats
)
val_ds = Augmented(
    RxRxDataset(valid, NUM_CLASSES, onehot=True), 
    tr=transforms,
    norm_fn=multichannel_norm,
    norm_stats=stats
)
loaders = OrderedDict([
    ('train', DataLoader(trn_ds, batch_size=batch_size, num_workers=12, shuffle=True)),
    ('valid', DataLoader(val_ds, batch_size=batch_size, num_workers=12, shuffle=False))
])

Train experiments: ['HEPG2-01', 'HEPG2-02', 'HEPG2-03', 'HEPG2-04', 'HEPG2-05', 'HEPG2-07', 'HUVEC-01', 'HUVEC-02', 'HUVEC-03', 'HUVEC-04', 'HUVEC-05', 'HUVEC-06', 'HUVEC-07', 'HUVEC-08', 'HUVEC-09', 'HUVEC-10', 'HUVEC-11', 'HUVEC-13', 'HUVEC-14', 'HUVEC-15', 'HUVEC-16', 'RPE-01', 'RPE-02', 'RPE-03', 'RPE-04', 'RPE-06', 'RPE-07', 'U2OS-02', 'U2OS-03']
Valid experiments: ['HEPG2-06', 'HUVEC-12', 'RPE-05', 'U2OS-01']
Train size: 64166, valid size: 8864


In [8]:
#export
def get_model_with_simple_custom_head(model_name, num_classes, pretrained='imagenet'):
    model_fn = pretrainedmodels.__dict__[model_name]
    model = model_fn(num_classes=1000, pretrained=pretrained)
    dim_feats = model.last_linear.in_features
    model.last_linear = nn.Linear(dim_feats, num_classes)
    conv0 = model.features.conv0
    new_conv = nn.Conv2d(6, 64, 7, 2, 3, bias=False)
    new_conv.weight.data[:, 0:3, :] = conv0.weight.data.clone()
    new_conv.weight.data[:, 3:6, :] = conv0.weight.data.clone()
    model.features.conv0 = new_conv
    del conv0
    return model

In [8]:
#export
epochs = 1000
model = get_model_with_simple_custom_head('densenet121', NUM_CLASSES)
freeze_all(model)
unfreeze_layers(model, ['features.denseblock4.denselayer16', 'last_linear'])
opt = torch.optim.AdamW(
    params=[
        {'params': model.features.denseblock4.denselayer16.parameters(), 'lr': 1e-4},
        {'params': model.last_linear.parameters(), 'lr': 1e-3}
    ],
    weight_decay=0.01
)
sched = CosineAnnealingWarmRestarts(opt, T_0=len(loaders['train']), T_mult=2, eta_min=1e-6)
loss_fn = nn.CrossEntropyLoss()
runner = SupervisedRunner()

Unfreezing layer features.denseblock4.denselayer16
Unfreezing layer last_linear


In [ ]:
#export
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [10]:
#export
runner.train(
    model=model,
    num_epochs=epochs,
    criterion=loss_fn,
    optimizer=opt,
    scheduler=sched,
    logdir='/tmp/exp_split/',
    loaders=loaders,
    callbacks=[
        AccuracyCallback(num_classes=NUM_CLASSES),
        EarlyStoppingCallback(10, metric='accuracy01', minimize=False),
        BatchMetricsPlotCallback(use_env_creds=True),
        EpochMetricsPlotCallback(use_env_creds=True)
    ],
    verbose=True
)

W0912 01:11:15.121436 140092134479680 __init__.py:505] Setting up a new session...
W0912 01:11:15.228921 140092134479680 __init__.py:505] Setting up a new session...


Early exiting                                                                                                                                                   
0/1000 * Epoch (train):  14% 18/126 [00:39<02:20,  1.30s/it, _timers/_fps=2182.449, accuracy01=0.391, accuracy03=0.586, accuracy05=0.781, loss=7.232]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f692d746598>
Traceback (most recent call last):
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/home/ck/anaconda3/envs/fastai_10/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


## Test

In [10]:
data_dict = load_data()
_, test = data_dict['data']
tst_ds = Augmented(
    RxRxDataset(test, NUM_CLASSES, onehot=True), 
    tr=transforms,
    norm_fn=multichannel_norm,
    norm_stats=stats
)
tst_dl = DataLoader(tst_ds, batch_size=1024, shuffle=False, num_workers=12)